In [42]:
import pandas as pd;import numpy as np;import sys; 
from datetime import datetime
from datetime import timedelta


#降水量タームを時間に変換
def convert_precipitation_term_org_to_hour(df, col_precipitation_term):
    df[col_precipitation_term] = df[col_precipitation_term].replace(
        {1 : 6,
         2 : 12,
         3 : 18,
         4 : 24,
         5 : 1,
         6 : 2,
         7 : 3,
         8 : 9,
         9 : 15,
         -999999999 : 0})
    return df

#降水量オリジナル(990以上)をmm/hに変換
#990は微量という表現で、降水なしと区別するため、便宜的に0.001とする
def convert_precipitation_org_to_mm(df, col_precipitation):
    df[col_precipitation] = df[col_precipitation].replace(
        {990 : 0.001,
         991 : 0.1,
         992 : 0.2,
         993 : 0.3,
         994 : 0.4,
         995 : 0.5,
         996 : 0.6,
         997 : 0.7,
         998 : 0.8,
         999 : 0.9,         
         -999999999 : 0})
    return df  
    
    
""" 
   #PrecTermは過去◯時間の合計降水量の◯時間部分
   #precは過去◯時間の合計降水量

   id,utc,Precipitation,PrecipitationTerm
   47662,2011/06/01-00:00:00,0,2
   47662,2011/06/01-03:00:00,-999999999,-999999999
   47662,2011/06/01-06:00:00,990,1
   47662,2011/06/01-09:00:00,-999999999,-999999999
   47662,2011/06/01-12:00:00,990,2
   47662,2011/06/01-15:00:00,-999999999,-999999999
   47662,2011/06/01-18:00:00,990,1
   47662,2011/06/01-21:00:00,-999999999,-999999999
   47662,2011/06/02-00:00:00,2,2
   47662,2011/06/02-03:00:00,-999999999,-999999999
   47662,2011/06/02-06:00:00,4,1
   47662,2011/06/02-09:00:00,-999999999,-999999999
   47662,2011/06/02-12:00:00,8,2
   47662,2011/06/02-15:00:00,-999999999,-999999999
   47662,2011/06/02-18:00:00,995,1
   47662,2011/06/02-21:00:00,-999999999,-999999999
   47662,2011/06/03-00:00:00,995,2
   47662,2011/06/03-03:00:00,-999999999,-999999999
   47662,2011/06/03-06:00:00,0,1
"""
df_org = pd.read_csv("/usr/amoeba/pub/for_intern/data/1001_processed_data.csv")
#df_org = pd.read_csv("/usr/amoeba/pub/for_intern/data/test.csv")
df_org.head(3)

,id,Temperature,Precipitation,PrecipitationTerm,ID,utc_offset_hours,localtime
0,1001,-51,0.1,6,1001,2.0,2011-01-31 14:00:00
1,1001,-61,0.1,12,1001,2.0,2011-01-31 08:00:00
2,1001,-23,0.0,6,1001,2.0,2011-01-31 02:00:00


In [ ]:

#降水量タームを時間に変換
#df_org = convert_precipitation_term_org_to_hour(df_org,"PrecipitationTerm")
#降水量オリジナル(990以上)をmm/hに変換
#df_org = convert_precipitation_org_to_mm(df_org,"Precipitation")

#df_org = df_org.rename(columns={'Precipitation': 'prec'})
#df_org = df_org.rename(columns={'PrecipitationTerm': 'PrecTerm'})


#prectermが0のデータを削除
df_org = df_org[df_org['PrecipitationTerm']!=0]

df_org["localtime"] = pd.to_datetime(df_org["localtime"])

utc_list = sorted(df_org["localtime"].value_counts().index)

df_org = df_org.sort_values(by=["localtime"], ascending=True)


df_all = pd.DataFrame()

for (i,utc) in enumerate(utc_list,1):

   if i == 1:

      df1 = df_org[df_org["localtime"]==utc]

      hour_interval1 = df1["PrecipitationTerm"].iloc[0]

      total_prec1 = df1["Precipitation"].iloc[0]

      prec_1hour = total_prec1/hour_interval1

      first_data = [] 
      
      for i in range(0,hour_interval1):
         
         valid_utc = utc + timedelta(hours=-i)
         
         first_data.append([valid_utc,prec_1hour])
         
      df_all = pd.DataFrame(first_data).rename(columns={0:"localtime",1:"Precipitation"})
      
   else:
      end_utc2 = utc
      
      df_second = df_org[df_org["localtime"]==end_utc2]

      hour_interval2 = df_second["PrecipitationTerm"].iloc[0]

      start_utc2 = utc - timedelta(hours = int(hour_interval2))
      
      include_before_data = df_all[(df_all["localtime"]>start_utc2)&(df_all["localtime"]<=end_utc2)]

      
      if len(include_before_data)>0:

         include_before_total_prec = include_before_data["Precipitation"].sum()

         total_prec2 = df_second["Precipitation"].iloc[0]

         before_end_time = include_before_data["localtime"].max()

         time_interval2 = int((end_utc2 - before_end_time)/np.timedelta64(1,'h'))

         value_timestep2 = (total_prec2 - include_before_total_prec)/time_interval2

         second_data = []

         for i in range(0,time_interval2):

            utc2 = end_utc2 + timedelta(hours = -i)

            second_data.append([utc2, value_timestep2])
            
      elif len(include_before_data)==0:
         
         include_total_prec = 0

         total_prec2 = df_second["Precipitation"].iloc[0]

         time_interval2 = hour_interval2

         value_timestep2 = (total_prec2 - include_total_prec)/time_interval2

         second_data = []

         for i in range(0,time_interval2):

            utc2 = end_utc2 + timedelta(hours=-i)

            second_data.append([utc2, value_timestep2])
            
      df_second_data = pd.DataFrame(second_data).rename(columns={0:"localtime",1:"Precipitation"})
         
      df_all = pd.concat([df_second_data, df_all]).sort_values(by=["localtime"], ascending=True)
      
#作成したファイルをローカルに保存     
df_all.to_csv('/usr/amoeba/pub/for_intern/data/test_after_divided_to_hourly.csv', header=True, index=False, mode="w")      
   

,localtime,Precipitation
5,2015-12-28 09:00:00,0.016667
4,2015-12-28 10:00:00,0.016667
3,2015-12-28 11:00:00,0.016667
2,2015-12-28 12:00:00,0.016667
1,2015-12-28 13:00:00,0.016667
0,2015-12-28 14:00:00,0.016667
5,2015-12-28 15:00:00,0.316667
4,2015-12-28 16:00:00,0.316667
3,2015-12-28 17:00:00,0.316667
2,2015-12-28 18:00:00,0.316667


,id,Temperature,Precipitation,PrecipitationTerm,ID,utc_offset_hours,localtime
116,1001,-56,0.0,6,1001,2.0,2011-01-01 02:00:00
115,1001,-52,0.0,6,1001,2.0,2011-01-01 14:00:00
114,1001,-48,0.0,12,1001,2.0,2011-01-01 20:00:00
113,1001,-28,0.0,6,1001,2.0,2011-01-02 02:00:00
112,1001,-22,0.0,12,1001,2.0,2011-01-02 08:00:00
...,...,...,...,...,...,...,...
6748,1001,42,6.0,12,1001,2.0,2015-12-30 20:00:00
6747,1001,34,0.0,6,1001,2.0,2015-12-31 02:00:00
6746,1001,36,0.0,12,1001,2.0,2015-12-31 08:00:00
6745,1001,35,0.0,6,1001,2.0,2015-12-31 14:00:00
